# IsoNet
------
This will be used to create the model for predicting precipitable stable water isotopes. Only creates and exports the model and trains it. Is not used to predict.

Table of Contents:
1. [Importing Data](#importing-data)
2. [Data Preprocessing](#data-preprocessing)
3. [Model Creation](#model-creation)
4. [Model Training](#model-training)
5. [Model Export](#model-export)

In [69]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import datetime
import matplotlib.pyplot as plt

# Tensorflow and keras libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError

## Importing Data
Importing the data, and then splitting it into features and labels

In [70]:
# Load in the data, separate the features and labels
data = pd.read_csv(r'Isoscape_Data.csv')
features = data.drop(['H2avg', 'dex', 'O18Avg', 'Station'], axis=1)
labels = data['O18Avg']

# Adjusting the date time to be int values where Janurary 1st, of year 1 is 1
features['Date'] = pd.to_datetime(features['Date'])
features['Date'] = features['Date'].map(datetime.datetime.toordinal)

numFeatures = features.shape[1]

## Data Preprocessing
Preprocessing the data, including:
* Convert into numpy arrays
* Split into training and testing sets
* Convert into tensorflow Datasets
* Windowing the data (Skipping for now, will do later) 
* Standardizing the data

In [71]:
# Split the data into training and testing sets
splitIndex = int(0.8 * len(features))
xTrain = features[:splitIndex]
xTest = features[splitIndex:]
yTrain = labels[:splitIndex]
yTest = labels[splitIndex:]

# Scale the data
scaler = StandardScaler()
xTrain = scaler.fit_transform(xTrain)
xTest = scaler.transform(xTest)

# Convert the data into numpy arrays
xTrain = np.array(xTrain)
yTrain = np.array(yTrain)
xVal = xTrain[:int(0.2 * len(xTrain))]
yVal = yTrain[:int(0.2 * len(yTrain))]
xTrain = xTrain[int(0.2 * len(xTrain)):]
yTrain = yTrain[int(0.2 * len(yTrain)):]

xTest = np.array(xTest)
yTest = np.array(yTest)


# Convert traindata into tensor dataset
trainData = tf.data.Dataset.from_tensor_slices((xTrain, yTrain))
valData = tf.data.Dataset.from_tensor_slices((xVal, yVal))

## Model Creation
Creating the model, including:
* Creating the model architecture
* Compiling the model

Model architecture:
* Input layer
* 1 LSTM layer
* 2 Dense layers
* Output layer

In [72]:
numNeuorns = 64
model = Sequential()
model.add(InputLayer(input_shape=(numFeatures, 1)))
model.add(LSTM(numNeuorns))
model.add(Dense(numNeuorns, activation='relu'))
model.add(Dense(numNeuorns, activation='relu'))
model.add(Dense(1))

model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse', metrics=[RootMeanSquaredError(), MeanAbsoluteError()])

## Model Training
Training and evaluating the model

In [73]:
es = EarlyStopping(monitor='val_loss', mode = 'min', patience=100, restore_best_weights=True)

batchSize = 64
model.fit(
    trainData.batch(batchSize),
    batch_size=batchSize,
    epochs=500,
    validation_data=valData.batch(batchSize),
    callbacks=[es]
)

Epoch 1/500


18/18 [==============================] - 3s 38ms/step - loss: 306.9258 - root_mean_squared_error: 17.5193 - mean_absolute_error: 15.8370 - val_loss: 719.3087 - val_root_mean_squared_error: 26.8199 - val_mean_absolute_error: 25.9037
Epoch 2/500
18/18 [==============================] - 0s 9ms/step - loss: 305.9501 - root_mean_squared_error: 17.4914 - mean_absolute_error: 15.8052 - val_loss: 717.7139 - val_root_mean_squared_error: 26.7902 - val_mean_absolute_error: 25.8737
Epoch 3/500
18/18 [==============================] - 0s 9ms/step - loss: 304.8859 - root_mean_squared_error: 17.4610 - mean_absolute_error: 15.7711 - val_loss: 715.6748 - val_root_mean_squared_error: 26.7521 - val_mean_absolute_error: 25.8355
Epoch 4/500
18/18 [==============================] - 0s 8ms/step - loss: 303.5634 - root_mean_squared_error: 17.4231 - mean_absolute_error: 15.7292 - val_loss: 713.1160 - val_root_mean_squared_error: 26.7042 - val_mean_absolute_error: 25.7874
Epoch 5/500
18/18 [====================

## Model Export

In [74]:
# Printing how well the model did on the test data
print(model.evaluate(xTest, yTest))

12/12 [==============================] - 1s 4ms/step - loss: 15.5613 - root_mean_squared_error: 3.9448 - mean_absolute_error: 3.0053
[15.561327934265137, 3.9447848796844482, 3.0052924156188965]


In [75]:
# Save the model
model.save('Models/UN-NAMED.keras')

In [35]:
# Create new pd DataFrame with the predictions and actual values including the features
predictions = model.predict(xTest)
df = features.loc[splitIndex:]
df['Predictions'] = predictions
df['Actual'] = yTest

# Export the data to a csv file
df.to_csv('results_test.csv', index=False)

12/12 [==============================] - 0s 3ms/step


/tmp/ipykernel_5119/3509207634.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Predictions'] = predictions
/tmp/ipykernel_5119/3509207634.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Actual'] = yTest


In [37]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 64)                16896     
                                                                 
 dense_9 (Dense)             (None, 64)                4160      
                                                                 
 dense_10 (Dense)            (None, 64)                4160      
                                                                 
 dense_11 (Dense)            (None, 1)                 65        
                                                                 
Total params: 25281 (98.75 KB)
Trainable params: 25281 (98.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
